In [1]:
import nlopt
from cpg_fsm import *
from numpy import *

In [2]:
loop_freq = 100  # Hz
FSM = FiniteStateMachine(loop_freq)
t = time.time()
iter = 0

In [3]:
def getTrajectoryCost(bezier_profiles, grad):
    # if grad.size > 0:
    global iter
    global t
    print("t", time.time()-t)
    t = time.time()
    bp = bezier_profiles.reshape(4, -1)
    print(bp)
    CORGI = Corgi(loop_freq, FSM)
    CORGI.standUp(0.8)
    CORGI.move(swing_profile=bp)
    print("iter", iter,"cost",CORGI.cost)
    iter += 1
    return CORGI.cost

In [4]:
opt = nlopt.opt(nlopt.GN_CRS2_LM, 24)

step_height = 0.05
bnd_H = (step_height, 0.85)
bnd_dH = (0, 0.02)

f_bnd_L1 = (0, 0.15)
f_bnd_L2 = (-0.13, 0.1)
f_bnd_L3 = (0, 0.9)
f_bnd_L4 = (-0.13, 0.1)

h_bnd_L1 = (0, 0.1)
h_bnd_L2 = (-0.25, 0.1)
h_bnd_L3 = (0, 0.03)
h_bnd_L4 = (-0.1, 0.1) 

bez_prof_init = np.array([[0.07, 0.01, 0.15, 0.05, 0.08, 0.06],
                          [0.07, 0.01, 0.15, 0.05, 0.08, 0.06],
                          [0.07, 0.01, 0.08, 0.05, 0.02, 0.1],
                          [0.07, 0.01, 0.08, 0.05, 0.02, 0.1]])
bez_prof_init = bez_prof_init.reshape(1, -1)[0]


opt.set_lower_bounds([step_height, 0, 0, -0.13, 0, -0.13,
                      step_height, 0, 0, -0.13, 0, -0.13,
                      step_height, 0, 0, -0.25, 0, -0.1,
                      step_height, 0, 0, -0.25, 0, -0.1])
opt.set_upper_bounds([0.85, 0.02, 0.15, 0.1, 0.9, 0.1,
                      0.85, 0.02, 0.15, 0.1, 0.9, 0.1,
                      0.85, 0.02, 0.1, 0.1, 0.03, 0.1,
                      0.85, 0.02, 0.1, 0.1, 0.03, 0.1,])
opt.set_min_objective(getTrajectoryCost)
# opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad,2,0), 1e-8)
# opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad,-1,1), 1e-8)
# opt.set_xtol_rel(1e-4)
opt.set_maxeval(1000)
x = opt.optimize(bez_prof_init)
minf = opt.last_optimum_value()

print("optimum at ", x)
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

t 0.03376626968383789
[[0.07 0.01 0.15 0.05 0.08 0.06]
 [0.07 0.01 0.15 0.05 0.08 0.06]
 [0.07 0.01 0.08 0.05 0.02 0.1 ]
 [0.07 0.01 0.08 0.05 0.02 0.1 ]]
cost -2.567192759867298
iter 0 cost -2.567192759867298
t 0.17836546897888184
[[ 0.0792441   0.00931504  0.12516066 -0.08390495  0.55942376 -0.10681562]
 [ 0.05661457  0.01189888  0.07510523  0.03165522  0.02327831  0.00798734]
 [ 0.39752347  0.0063899   0.02545244  0.05325076  0.0285877  -0.08197134]
 [ 0.54740484  0.01264693  0.07322042  0.0658941   0.02182554  0.04540328]]
p_hip [[0.2545  0.16475 0.196  ]]
swing_point [[0.57923125 0.16475    0.07889143]]
t 0.0321657657623291
[[ 0.43130735  0.00526264  0.13919265  0.08140228  0.50411164  0.04173093]
 [ 0.74556537  0.00370711  0.06340377 -0.08883036  0.76481071 -0.11094336]
 [ 0.44791235  0.00527151  0.08794402 -0.24757557  0.02409539  0.09246372]
 [ 0.44315813  0.01659699  0.02006096  0.09285609  0.02229435 -0.08836761]]
p_hip [[-0.202   -0.16475  0.196  ]]
swing_point [[-0.21874684

Traceback (most recent call last):
  File "/home/guanlunlu/MBD_Corgi_python/script/cpg_fsm.py", line 330, in move
    self.legs[i].moveSwing(lf_sp)
  File "/home/guanlunlu/MBD_Corgi_python/script/cpg_fsm.py", line 129, in moveSwing
    tb_ = lk.InverseKinematicsPoly(sp)
  File "/home/guanlunlu/MBD_Corgi_python/script/LegKinematics.py", line 149, in InverseKinematicsPoly
    raise ValueError("Desired length exceed ", str(desired_length), "Point desired", str(point_G.T))
ValueError: ('Desired length exceed ', '0.09360965682288597', 'Point desired', '[[-0.09290295 -0.01148087]]')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_30319/1913989593.py", line 11, in getTrajectoryCost
    CORGI.move(swing_profile=bp)
  File "/home/guanlunlu/MBD_Corgi_python/script/cpg_fsm.py", line 334, in move
    self.legs[i].moveSwing(lf_sp)
  File "/home/guanlunlu/MBD_Corgi_python/script/cpg_fsm.py", line 129, in moveSwing
    tb